In [1]:
import pandas as pd
import pickle
import sklearn as sk
from sklearn import ensemble
from sklearn import model_selection
from sklearn import metrics
import numpy as np

In [4]:
with open("data/adjacency_matrices.pkl", "rb") as file:
    adj = pickle.load(file)

In [27]:
xTrain = pd.read_csv("data/input_train.csv", index_col=0)
yTrain = pd.read_csv("data/y_train.csv", index_col=0)

In [28]:
xTrain['sample_id'] = 21 * xTrain['cluster'] + xTrain['day']
xTrain.set_index('sample_id', inplace=True)
xTrain.sort_index(inplace=True)

In [29]:
xTrain

,cluster,day,asset,md,bc,ret_1,ret_2,ret_3,ret_4,ret_5,...,ret_14,ret_15,ret_16,ret_17,ret_18,ret_19,ret_20,ret_21,ret_22,ret_23
sample_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,16,-5.945457,0.704236,-0.004318,0.006388,-0.001805,0.000525,-0.007697,...,0.002379,0.001958,-0.001539,-0.003084,0.010648,0.005768,0.007257,0.007262,-0.003634,0.007815
0,0,0,7,-5.609071,0.674829,0.013302,0.009210,-0.002413,0.012408,0.014749,...,0.005435,-0.004769,0.010330,-0.004743,0.009320,0.017104,0.004952,0.009137,0.005697,0.020635
0,0,0,18,-5.985903,0.622846,-0.001495,0.003743,0.016406,-0.019809,-0.002994,...,-0.012177,NaN,0.005393,0.004598,0.000763,0.005335,0.004549,0.007547,-0.002996,0.010518
0,0,0,4,-5.601894,0.750745,0.001106,NaN,0.006074,-0.015917,-0.006693,...,-0.005137,-0.004590,0.010951,-0.003991,0.008586,0.007946,0.004505,0.006726,0.003898,0.010538
0,0,0,2,-5.482508,0.570504,0.002667,-0.001330,0.001108,-0.006589,-0.003343,...,0.005091,-0.003490,0.006132,-0.006076,0.002006,0.006751,0.000999,0.007497,0.003570,0.007921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30743,1463,20,5,-8.447019,0.271287,-0.003805,-0.011009,-0.002272,0.008424,-0.010386,...,-0.004009,-0.001565,0.020157,0.012075,-0.016703,0.005515,0.021501,-0.031787,-0.015750,0.011494
30743,1463,20,4,-8.219044,0.504312,-0.014207,-0.023265,0.001265,-0.004211,0.003383,...,-0.015017,0.008563,0.007869,-0.009246,0.004562,-0.009496,NaN,0.010004,-0.002064,NaN
30743,1463,20,0,-4.214598,0.219784,-0.005658,-0.013850,0.004954,0.001950,0.005352,...,-0.003811,-0.009191,0.008365,-0.004892,0.011489,-0.001902,0.002541,-0.000739,0.000423,0.002112


In [32]:
test1 = xTrain.groupby('sample_id').mean().drop(columns=['cluster', 'day', 'asset']).dropna()
test1['target'] = yTrain['target']
test1

,md,bc,ret_1,ret_2,ret_3,ret_4,ret_5,ret_6,ret_7,ret_8,...,ret_15,ret_16,ret_17,ret_18,ret_19,ret_20,ret_21,ret_22,ret_23,target
sample_id,,,,,,,,,,,,,,,,,,,,,
0,-5.705550,0.599596,-0.003180,0.001928,0.002466,-0.010499,0.001530,-0.003849,-0.006318,-0.017592,...,-0.002826,0.005674,-0.005571,0.006949,0.005116,0.003103,0.009894,0.002359,0.006052,0.000301
1,-5.705550,0.599596,0.013303,0.001179,0.001234,-0.013514,0.014452,0.007980,-0.005909,0.003051,...,-0.004987,0.013930,0.006389,0.003227,0.005479,-0.021580,-0.011090,0.005877,0.001482,-0.011966
2,-5.705550,0.599596,-0.012115,0.023476,0.010832,0.004019,-0.000939,-0.015029,-0.003969,0.007315,...,-0.009952,-0.012736,-0.008211,0.009511,0.001003,-0.012755,-0.020219,-0.011961,0.009295,-0.011395
3,-5.705550,0.599596,-0.008271,0.007669,-0.002567,-0.001642,0.000680,0.021091,-0.002882,-0.015067,...,0.019252,0.013038,0.006508,0.000776,-0.012990,-0.014617,-0.011488,0.004350,0.018642,0.000664
4,-5.705550,0.599596,0.011887,0.001898,-0.010022,-0.013176,0.003421,0.008563,0.002711,0.000798,...,0.004529,0.004875,-0.004420,0.004123,0.019168,-0.001895,-0.001574,0.000908,0.005075,0.002852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30739,-6.751337,0.471494,-0.024104,-0.016507,-0.005770,0.008415,-0.001305,-0.001000,-0.017745,-0.016250,...,0.000912,-0.002110,0.000982,-0.003174,0.000514,0.002982,0.013443,0.010602,-0.002606,0.014456
30740,-6.751337,0.471494,-0.007096,-0.012172,-0.001522,-0.007782,0.003664,0.008472,0.001113,0.003364,...,0.010003,0.006559,0.001790,-0.000665,-0.004928,-0.008155,0.008358,-0.011834,-0.006234,0.001465
30741,-6.751337,0.471494,0.008057,0.003378,0.001637,0.007419,-0.004770,-0.006124,-0.009467,-0.002242,...,0.000453,-0.001815,-0.000434,0.003667,0.003312,0.027649,0.005071,0.004436,-0.004398,-0.001114


In [33]:
test1.to_csv('data/test1.csv')

In [34]:
xTest = pd.read_csv('data/input_test.csv')
xTest

,id,cluster,day,asset,md,bc,ret_1,ret_2,ret_3,ret_4,...,ret_14,ret_15,ret_16,ret_17,ret_18,ret_19,ret_20,ret_21,ret_22,ret_23
0,491038,1464,13,7,-5.422238,0.005959,0.033248,-0.011551,0.011686,-0.009901,...,0.044199,0.022487,-0.048512,0.021074,-0.007989,0.018121,0.009888,-0.033290,0.022957,0.017162
1,491039,1464,15,3,-4.944505,0.004303,0.003802,-0.016856,-0.000193,0.001927,...,0.010023,0.005725,-0.009108,-0.003638,0.001345,0.046065,-0.040367,0.001912,NaN,-0.013168
2,491040,1464,14,3,-4.944505,0.004303,-0.009852,0.017987,-0.006579,0.013245,...,-0.013133,0.011407,0.007519,0.009328,-0.016636,-0.013158,-0.001905,-0.003817,0.005747,0.011429
3,491041,1464,13,3,-4.944505,0.004303,0.003891,0.001938,0.015474,-0.005714,...,-0.006106,-0.026744,0.010954,0.021120,-0.011331,-0.028743,0.025098,-0.046592,-0.015332,0.010705
4,491042,1464,12,3,-4.944505,0.004303,0.024691,0.028112,0.003906,-0.021401,...,0.004536,0.056352,-0.033457,-0.007308,0.007361,0.001923,-0.007678,-0.023211,0.030891,-0.024203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217902,708940,2090,17,3,-7.335009,0.658364,-0.008482,0.008104,0.001340,-0.000446,...,-0.015578,-0.008352,0.000887,-0.019486,-0.023487,-0.017114,0.011765,-0.020465,-0.009497,0.012464
217903,708941,2090,18,3,-7.335009,0.658364,-0.000964,0.016884,-0.015655,-0.009639,...,-0.011759,0.025781,-0.015950,0.012279,-0.003882,-0.007793,0.006382,0.007317,0.009201,0.011036
217904,708942,2090,19,3,-7.335009,0.658364,0.009259,0.008691,-0.002393,-0.007198,...,0.003498,-0.005478,0.018528,-0.002950,-0.004931,0.008424,-0.005405,0.011364,0.001466,-0.012195
217905,708943,2090,0,4,-7.394441,0.642147,0.004223,0.005887,-0.005853,-0.000841,...,-0.004227,0.001698,0.004237,0.005063,-0.004198,-0.007589,0.002549,-0.002542,-0.011045,0.024055


In [36]:
xTest['sample_id'] = 21 * xTest['cluster'] + xTest['day']
xTest

,id,cluster,day,asset,md,bc,ret_1,ret_2,ret_3,ret_4,...,ret_15,ret_16,ret_17,ret_18,ret_19,ret_20,ret_21,ret_22,ret_23,sample_id
0,491038,1464,13,7,-5.422238,0.005959,0.033248,-0.011551,0.011686,-0.009901,...,0.022487,-0.048512,0.021074,-0.007989,0.018121,0.009888,-0.033290,0.022957,0.017162,30757
1,491039,1464,15,3,-4.944505,0.004303,0.003802,-0.016856,-0.000193,0.001927,...,0.005725,-0.009108,-0.003638,0.001345,0.046065,-0.040367,0.001912,NaN,-0.013168,30759
2,491040,1464,14,3,-4.944505,0.004303,-0.009852,0.017987,-0.006579,0.013245,...,0.011407,0.007519,0.009328,-0.016636,-0.013158,-0.001905,-0.003817,0.005747,0.011429,30758
3,491041,1464,13,3,-4.944505,0.004303,0.003891,0.001938,0.015474,-0.005714,...,-0.026744,0.010954,0.021120,-0.011331,-0.028743,0.025098,-0.046592,-0.015332,0.010705,30757
4,491042,1464,12,3,-4.944505,0.004303,0.024691,0.028112,0.003906,-0.021401,...,0.056352,-0.033457,-0.007308,0.007361,0.001923,-0.007678,-0.023211,0.030891,-0.024203,30756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217902,708940,2090,17,3,-7.335009,0.658364,-0.008482,0.008104,0.001340,-0.000446,...,-0.008352,0.000887,-0.019486,-0.023487,-0.017114,0.011765,-0.020465,-0.009497,0.012464,43907
217903,708941,2090,18,3,-7.335009,0.658364,-0.000964,0.016884,-0.015655,-0.009639,...,0.025781,-0.015950,0.012279,-0.003882,-0.007793,0.006382,0.007317,0.009201,0.011036,43908
217904,708942,2090,19,3,-7.335009,0.658364,0.009259,0.008691,-0.002393,-0.007198,...,-0.005478,0.018528,-0.002950,-0.004931,0.008424,-0.005405,0.011364,0.001466,-0.012195,43909
217905,708943,2090,0,4,-7.394441,0.642147,0.004223,0.005887,-0.005853,-0.000841,...,0.001698,0.004237,0.005063,-0.004198,-0.007589,0.002549,-0.002542,-0.011045,0.024055,43890


In [43]:
test2 = xTest.drop(columns=['id','cluster','day','asset']).groupby('sample_id').mean().fillna(0)
test2.to_csv('data/test2.csv')

In [2]:
test1 = pd.read_csv('data/test1.csv').set_index('sample_id')
test2 = pd.read_csv('data/test2.csv').set_index('sample_id')
X, Y = test1.drop(columns='target'), test1['target']

In [3]:
randomForest = ensemble.RandomForestRegressor()
gradientBoosting = ensemble.GradientBoostingRegressor()
extraTrees = ensemble.ExtraTreesRegressor()

In [42]:
xTrain, xTest, yTrain, yTest = model_selection.train_test_split(X, Y, test_size=.25)

In [35]:
param_grid = {
    'n_estimators': [10],
    'criterion': ["squared_error", "absolute_error"],
    # 'max_depth': [2,8,16,32,50],
    # 'min_sample_split': [2,4,6],
    # 'min_sample_leaf': [1,2],
    #'oob_score': [True, False],
    # 'max_features': ['auto', None],    
    # 'bootstrap': [True, False],
    # 'warm_start': [True, False]
}

param_grid_RF = {
    'n_estimators': [100],
    # 'verbose': [1],
    # 'criterion': ["squared_error", "absolute_error"],
    # 'max_depth': [2,8,16,32,50],
    # 'min_sample_split': [2,4,6],
    # 'min_sample_leaf': [1,2],
    #'oob_score': [True, False],
    'max_features': ['auto', 'sqrt', 'log2', None],    
    # 'bootstrap': [True, False],
    # 'warm_start': [True, False]
}

param_grid_GB = {
    'n_estimators': [100],
    # 'verbose': [1],
    'criterion': ["squared_error"],
    # 'max_depth': [2,8,16,32,50],
    # 'min_sample_split': [2,4,6],
    # 'min_sample_leaf': [1,2],
    #'oob_score': [True, False],
    # 'max_features': ['auto', 'sqrt', 'log2', None],    
    # 'bootstrap': [True, False],
    # 'warm_start': [True, False]
}

param_grid_ET = {
    'n_estimators': [100],
    # 'verbose': [1],
    # 'criterion': ["squared_error"],
    # 'max_depth': [2,8,16,32,50,None],
    # 'min_samples_split': [2,4,6,None],
    # 'min_sample_leaf': [1,2],
    #'oob_score': [True, False],
    # 'max_features': ['auto', 'sqrt', 'log2', None],    
    # 'bootstrap': [True, False],
    # 'warm_start': [True, False]
}

In [36]:
gcv = model_selection.GridSearchCV(extraTrees, cv=2, param_grid=param_grid_ET, n_jobs=-1, verbose=2, scoring="neg_root_mean_squared_error").fit(X, Y)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] END ...........min_samples_split=None, n_estimators=100; total time=   0.1s
[CV] END ...........min_samples_split=None, n_estimators=100; total time=   0.0s
[CV] END ..............min_samples_split=6, n_estimators=100; total time=  11.8s
[CV] END ..............min_samples_split=4, n_estimators=100; total time=  14.4s
[CV] END ..............min_samples_split=4, n_estimators=100; total time=  15.5s
[CV] END ..............min_samples_split=6, n_estimators=100; total time=  16.5s
[CV] END ..............min_samples_split=2, n_estimators=100; total time=  17.6s
[CV] END ..............min_samples_split=2, n_estimators=100; total time=  18.7s


/home/lwi/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
2 fits failed out of a total of 8.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/home/lwi/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/lwi/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 442, in fit
    trees = Parallel(
  File "/home/lwi/.local/lib/python3.8/site-packages/joblib/parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "/home/lwi/.local/lib/python3.8/site-pack

In [37]:
pd.DataFrame(gcv.cv_results_).drop(columns=["params", "std_fit_time", "mean_score_time",	"std_score_time", "split0_test_score", "split1_test_score"])

,mean_fit_time,param_min_samples_split,param_n_estimators,mean_test_score,std_test_score,rank_test_score
0,17.656135,2,100,-0.007474,0.000123,1
1,14.450240,4,100,-0.007504,0.000113,2
2,13.637656,6,100,-0.007635,0.000139,3
3,0.049374,None,100,NaN,NaN,4


In [ ]:
int(np.sqrt(metrics.mean_squared_error(yTest, gcv.predict(xTest))) * 10000)

In [55]:
reg = ensemble.ExtraTreesRegressor(verbose=1, n_estimators=200, n_jobs=-1)

In [56]:
reg.fit(xTrain, yTrain)

      Iter       Train Loss   Remaining Time 
         1           0.0001           37.40s
         2           0.0001           35.33s
         3           0.0001           34.11s
         4           0.0001           34.09s
         5           0.0001           33.62s
         6           0.0001           33.42s
         7           0.0001           33.09s
         8           0.0001           33.67s
         9           0.0001           33.24s
        10           0.0001           32.89s
        20           0.0001           29.82s
        30           0.0001           28.00s
        40           0.0001           26.03s
        50           0.0001           24.34s
        60           0.0001           22.74s
        70           0.0001           21.34s
        80           0.0001           19.87s
        90           0.0001           18.23s
       100           0.0001           16.61s
       200           0.0001            0.00s


GradientBoostingRegressor(n_estimators=200, verbose=1)

In [57]:
int(np.sqrt(metrics.mean_squared_error(yTest, reg.predict(xTest))) * 10000)

102